In [ ]:
%matplotlib inline
import IPython
import sys
sys.path.append('/home/localhorst87/Documents/coding/rl/finance/trading_rl/lib/') # edit with your lib path

from DataProvider import *
from Environment import *
from Learning import *
import Rewarder
import ActionSpace

import numpy as np
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import pdb
import math

In [ ]:
dataPath = '/home/localhorst87/Documents/coding/rl/finance/quant_hour_data.p' # edit with your data path
windowLength = 50

actionSpace = ActionSpace.SeparatedNets()
rewarder = Rewarder.WeightedMaxMinOpeningRewarder()
env = Environment(dataPath, windowLength, actionSpace, rewarder)

In [ ]:
tf.reset_default_graph()
agent = OpeningAgent(env)

In [ ]:
rewards = []
meanRewards = []
trainingLoss = []

In [ ]:
stepSizeTraining = 50
agent.batchSize = 256 # don't use small batches as we treat with a lot of noise in by optimizing a gaussian distribution!
agent.learningRate = 1e-4

In [ ]:
agent.minRewardTarget = 0.00
agent.probabilityThreshold = 0.50
agent.randomActionProbability = 1.00

In [ ]:
for _ in range(100000): 
    
    reward, action, loss = agent.runEpisode()
    
    rewards.append(reward)
    meanRewards.append(np.mean(rewards[-500:]))
    if loss is not None: trainingLoss.append(loss)

    if agent.episode > 10000 and agent.randomActionProbability >= 0.05:
        agent.randomActionProbability -= 0.95/40000

    if agent.episode > 10000 and agent.probabilityThreshold < 0.67:
        agent.probabilityThreshold += (0.67-0.50)/50000

    if agent.episode > 15000 and agent.minRewardTarget < 0.10:
        agent.minRewardTarget += 0.10/50000
    
    print(str(agent.episode) + ") minReward:" + str(round(agent.minRewardTarget,2)) + \
          " probThresh:" + str(round(agent.probabilityThreshold,2)) + \
          " randomProb:" + str(round(agent.randomActionProbability,2)) + " --> " + action)
    
    if agent.episode % 100 == 0:
        
        IPython.display.clear_output()
    
        plt.subplot(2,1,1)
        plt.plot(meanRewards[-5000:])
        plt.grid()

        plt.subplot(2,1,2)
        plt.plot(trainingLoss)
        plt.grid()

        plt.show()